In [1]:
import numpy as np
import math as m
import vice
from vice.yields.presets import JW20
import sys
sys.path.append("..")
from src import mcmc
print(sys.version_info)

/var/home/johnson.7419.local/VICE/vice/__init__.py:141: UserWarning: Using un-released version of VICE
  warnings.warn("Using un-released version of VICE", UserWarning)


sys.version_info(major=3, minor=10, micro=4, releaselevel='final', serial=0)


In [2]:
raw = np.genfromtxt("../mocksamples/noages.dat")
data = {}
data["[fe/h]"] = [row[0] for row in raw]
data["[fe/h]_err"] = [row[1] for row in raw]
data["[o/fe]"] = [row[2] for row in raw]
data["[o/fe]_err"] = [row[3] for row in raw]

In [34]:
class test(mcmc):
    
    def __init__(self, data):
        quantities = list(filter(lambda x: not x.endswith("_err"), data.keys()))
        self.sample = np.array([data[key] for key in quantities]).T
        self.errors = np.array([data["%s_err" % (key)] for key in quantities]).T
        super().__init__(data)
    
    def __call__(self, model):
        log_L_d_m = 0
        weights = np.random.rand(len(model))
        norm = sum(weights)
        weights = np.array([_ / norm for _ in weights])
        for i in range(len(self.sample)):
            L_di_m = 0
            cov = np.diag(self.errors[i]**2)
            invcov = np.linalg.inv(cov)
            for j in range(len(model)):
                delta = self.sample[i] - model[j]
                L_di_mj = weights[j] * m.exp(-0.5 * np.matmul(np.matmul(
                    delta, invcov), delta.T))
#                 L_di_mj = 1 / len(model) * m.exp(-0.5 * np.matmul(np.matmul(
#                     delta, invcov), delta.T))
                L_di_m += L_di_mj
            log_L_di_m = m.log(L_di_m)
            log_L_d_m += log_L_di_m
        print(log_L_d_m)
        
        self._fd.model = model
#         self._fd.weights = len(model) * [1 / len(model)]
        self._fd.weights = weights
        print(self._fd())

In [35]:
xvals = np.linspace(-1, 0, 100)
yvals = np.array([-0.2 * _ for _ in xvals])
model = np.array([xvals, yvals]).T
test_mcmc = test(data)
test_mcmc(model)

-18613.40363048271
-18613.40363048271
